# Simple Example of using the Pore and Surface Diffusion Model (PSDM)

Originally developed by Michigan Technological University, coded in Python by US Environmental Protection Agency

This can be used to model granular activated carbon (GAC) systems.

In [ ]:
## Import Required Packages
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import pylab as plt

import os
srt_dir = os.getcwd()

import PSDM.PSDM as PSDM
os.chdir(srt_dir)

## Files

This example uses "Example_TCE.xlsx" as its input file. 

Sheet "Properties" has chemical properties of compounds you wish to model. You can have more compounds in your Properties sheet than the model needs, so it may be helpful to create a standard sheet for numerous projects. 

Sheet "Kdata" has K [(ug/g)(L/ug)^(1/n)] & 1/n (unitless) information. 'brk' stores the days to complete breakthrough. 'q' is solid phase concentration (ug/g). 'AveC' is the average concentration (ug/L). This information can be calculated by the model for fitting data, but this provides a mechanism for storing/inputing these values for some simulations. 

Sheet "data" has a data structure for ['type', 'time', 'concentration', 'compound']. Where type corresponds to where samples were taken (e.g., 'influent', 'effluent'). 'Time' is numeric and should the maximum value determines the lengh of simulations (The units of 'Time' are listed in the "columnSpecs" sheet). 'Concentration' units are also listed in "columnSpecs". 'Compounds' should match those in "Properties", and should provide information for both influent and effluent data. 

Sheet "columnSpecs" has column information. 

An example of reading in these sheets is shown below.



In [ ]:
# =============================================================================
# Define the File to analyze
# can be in different folder, must be explicit for file location or can 
# use referential convention '../data/filename.xlsx' or 'C:/folders.../filename'
# =============================================================================
fn = 'Example_TCE.xlsx'

# =============================================================================
# READ IN Chemical Property Information
# may need to change location of file, depending on where it is stored locally
# properties are included in 'fn' from above, but can be separate file as well
# =============================================================================
chem_data = PSDM.process_input_data(fn, sheet_name='Properties') 
k_data = pd.read_excel(fn, sheet_name='Kdata',index_col=0) # K & 1/n data

# =============================================================================
# Read in Input File
# Can change location of data_sheet= and column_sheet= to match file naming
# =============================================================================
raw_data, column_info,\
compounds, carbons, = PSDM.process_input_file(fn,\
                                               data_sheet='data',\
                                               column_sheet='columnSpecs'
                                              )

# Example 1: Simple, no fouling

    chem_type = 'halogenated alkenes' #omitted, see Example 2 for how to use
    water_type = 'Organic Free'       #omitted, see Example 2 for how to use
    nr = radial collocation points
    nz = axial collocation points
    k_data = pandas dataframe containing K & 1/n data (ug/g)(L/ug)^(1/n) expected

In [ ]:
for comp in compounds:
    print(comp)
    
    # SETTING UP PSDM simulation column
    column = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data,
                   nr=8,
                   nz=12, 
                   k_data=k_data,\
                   optimize=False
                   )
    
    print(f'EBCT: {column.ebct:.2f} min') 
    column.test_range = np.array([k_data[comp]['K']]) 
    column.xn_range = np.array([k_data[comp]['1/n']])
    
    # RUNNING the simulation
    # only results is used in this example
    _compound, _k, _xn, _ssqs, results = column.run_psdm_kfit(comp)
    
    ### plotting results 
    # units are converted from ug/L to mg/L 
    plt.plot(results.index, 
             results.values/1000., 
             label='PSDM')
    (raw_data[column.influent][comp]/1000.).plot.line(label='influent',linestyle=':')
    
    # Plot Labels, etc.
    plt.legend()
    plt.title(f'{comp}\nK: {_k:,.2f} ($\mu$g/g)(L/$\mu$g)$^1$$^/$$^n$ - 1/n: {_xn:.3f}')
    plt.xlabel(column_info[carbons[0]]['time'])
    plt.ylabel('Concentration - mg/L (ppm)')
    plt.savefig(f'{comp}_{carbons[0]}_example1.png', dpi=300)
    
    
    tab2 = 'model-'+comp
    ## Set up file to write output
    with pd.ExcelWriter(f'Example_TCE_{carbons[0]}_example1.xlsx') as writer:
        results.to_excel(writer, sheet_name=tab2)



## Example 1a - Changing influent concentration programmatically  & Changing EBCT

In [ ]:
raw_data2 = raw_data.copy()              # create a copy of the raw data dataframe 
column_info2 = column_info[carbons[0]].copy()   # makes copy to change flowrate
## change the concentration associated with the 'influent' column
raw_data2[column.influent] = 5000        # 5ppm, rather than 50 ppm in original file



In [ ]:
for comp in compounds:
    print(comp)
    
    # SETTING UP PSDM simulation column
    column = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data2,
                   nr=8,
                   nz=12, 
                   k_data=k_data,\
                   optimize=False
                   )
    
    print(f'EBCT: {column.ebct:.2f} min')  
    column.test_range = np.array([k_data[comp]['K']])
    column.xn_range = np.array([k_data[comp]['1/n']])
    
    # RUNNING the simulation
    #only results is used in this example
    _compound, _k, _xn, _ssqs, results = column.run_psdm_kfit(comp)
    
    # Change Flowrate, which changes EBCT, and rerun simulation
    column_info2['flrt'] = column_info[carbons[0]]['flrt'] * 1.5 # internal units are mL/min, this just multiplies by 1.5
    column2 = PSDM.PSDM(column_info2, 
                   chem_data, 
                   raw_data2,
                   nr=8,
                   nz=12, 
                   k_data=k_data,\
                   optimize=False
                   )
    
    print(f'EBCT: {column2.ebct:.2f} min')  
    column2.test_range = np.array([k_data[comp]['K']])
    column2.xn_range = np.array([k_data[comp]['1/n']])
    _compound, _k, _xn, _ssqs, results2 = column2.run_psdm_kfit(comp)

    
    ### Plotting results 
    # plot results for 1st EBCT
    plt.plot(results.index, 
             results.values/1000., 
             label=f'EBCT: {column.ebct:.2f}')
    # plot results for 2nd EBCT
    plt.plot(results2.index, 
             results2.values/1000., 
             label=f'EBCT: {column2.ebct:.2f}')
    (raw_data2[column.influent][comp]/1000.).plot.line(label='influent',linestyle=':')
    
    # Plot Labels, etc.
    plt.legend()
    plt.title(f'{comp}\nK: {_k:,.2f} ($\mu$g/g)(L/$\mu$g)$^1$$^/$$^n$ - 1/n: {_xn:.3f}')
    plt.xlabel(column_info[carbons[0]]['time'])
    plt.ylabel('Concentration - mg/L (ppm)')
    plt.savefig(f'{comp}_{carbons[0]}_example1a.png', dpi=300)
    
    tab2 = 'model-'+comp
    with pd.ExcelWriter(f'Example_TCE_{carbons[0]}_example1a.xlsx') as writer:
        results.to_excel(writer, sheet_name=tab2)
    



# Example 2 - Variable influent

Variable influent data stored in "data_variable" sheet. Must reload info.
k_data didn't change, so will be reused from Example 1


In [ ]:
raw_data, column_info,\
compounds, carbons, = PSDM.process_input_file(fn,\
                                               data_sheet='data_variable',\
                                               column_sheet='columnSpecs'
                                              )
## User can look at raw_data to see new variable influent, or 
## see "data_variable" sheet


In [ ]:
chem_type = 'halogenated alkanes'
water_type = 'Rhine'
#note additional inputs in definition of column, below.

for comp in compounds:
    print(comp)
    
    # SETTING UP PSDM simulation column
    column = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data,
                   nr=8,
                   nz=12, 
                   k_data=k_data,
                   optimize=False
                   )
    
    print(f'EBCT: {column.ebct:.2f} min')
    column.test_range = np.array([k_data[comp]['K']])
    column.xn_range = np.array([k_data[comp]['1/n']])
    
    # RUNNING the simulation
    #only results is used in this example
    _compound, _k, _xn, _ssqs, results = column.run_psdm_kfit(comp)
    
    column2 = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data,
                   nr=8,
                   nz=12, 
                   k_data=k_data,
                   chem_type=chem_type,
                   water_type=water_type,
                   optimize=False
                   )
    
    column2.test_range = np.array([k_data[comp]['K']])
    column2.xn_range = np.array([k_data[comp]['1/n']])
    
    # RUNNING the simulation
    #only results is used in this example
    _compound, _k, _xn, _ssqs, results2 = column2.run_psdm_kfit(comp)
    
    ### plotting results 
    # units are converted from ug/L to mg/L 
    plt.plot(results.index, 
             results.values/1000., 
             label='PSDM-no fouling')
    plt.plot(results2.index, 
             results2.values/1000., 
             label='PSDM-with fouling')
    (raw_data[column.influent][comp]/1000.).plot.line(label='influent',linestyle=':')
    
    # Plot Labels, etc.
    plt.legend()
    plt.title(f'{comp}\nK: {_k:,.2f} ($\mu$g/g)(L/$\mu$g)$^1$$^/$$^n$ - 1/n: {_xn:.3f}')
    plt.xlabel(column_info[carbons[0]]['time'])
    plt.ylabel('Concentration - mg/L (ppm)')
    plt.savefig(f'{comp}_{carbons[0]}_example2.png', dpi=300)
    
    tab2 = 'model-'+comp
    
    with pd.ExcelWriter(f'Example_TCE_{carbons[0]}_example2.xlsx') as writer:
        results.to_excel(writer, sheet_name=tab2)



# Advanced Example

Demonstrating Fitting Capabilities.

Reloading all information for clarity.

Using "data_optimize" sheet, which contains time series of effluent breakthrough.

In [ ]:
#reset files, just in case
chem_data = PSDM.process_input_data(fn, sheet_name='Properties') 
k_data = pd.read_excel(fn, sheet_name='Kdata',index_col=0) # K & 1/n data


raw_data, column_info,\
compounds, carbons, = PSDM.process_input_file(fn,\
                                               data_sheet='data_optimize',\
                                               column_sheet='columnSpecs'
                                              )
## note data_sheet is now 'data_optimize'

chem_type = 'halogenated alkanes'
water_type = 'Rhine'

In [ ]:
## This step can take a few minutes
for comp in compounds:
    print(comp)
    column = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data,
                   nr=8,
                   nz=12, 
                #    k_data=k_data,
                   chem_type=chem_type,
                   water_type=water_type,
                   optimize=False
                   )
    
    print(column.k_data)

    column.test_range = np.array([k_data[comp]['K']])
    column.xn_range = np.array([k_data[comp]['1/n']])
    
    # RUNNING the simulation
    # only results is used in this example
    _compound, _k, _xn, _ssqs, results = column.run_psdm_kfit(comp)
    
    # print(results['data'][np.linspace(0,175,36)])
    
    plt.plot(results.index, 
             results.values/1000,':',
             label=f'True Case: K: {k_data[comp]["K"]:,.0f}, 1/n: {k_data[comp]["1/n"]}'
             )
    plt.plot(raw_data.index,
             raw_data[column_info[carbons[0]]['influentID']][comp].values/1000,
             label='influent',ls=':')
    plt.plot(raw_data.index,
             raw_data[carbons[0]][comp].values/1000.,
             label='effluent', marker='o',ls='--')
    
    
    column.xn_range = np.linspace(.3,.75,19) # increment 0.025
    # checks +/- 30% of estimated K (pm=30), 
    # and scans 1/n space in 0.01 step sizes (des_xn=0.01)
    column.run_all_smart(pm=30, des_xn=0.01)
    
    #Best fit was saved back to column.k_data
    print(column.k_data)
    
    _compound, k, xn, _ssqs, results2 = column.run_psdm_kfit(comp)
    
    plt.plot(results2.index,
             results2.values/1000.,
             label='Best Fit')
    
    
    # Plot Labels, etc.
    plt.legend()
    plt.title(f'{comp}\nK: {k:,.2f} ($\mu$g/g)(L/$\mu$g)$^1$$^/$$^n$ - 1/n: {xn:.3}')
    plt.xlabel(f'Time ({column_info.loc["time"].values[0]})')
    plt.ylabel('Concentration - mg/L (ppm)')
    plt.savefig(comp+'_'+carbons[0]+'_example3.png', dpi=300)


In [ ]:
## Updated Model Running Capability
## .model_uncertainty() will automatically run on a column object
column = PSDM.PSDM(column_info[carbons[0]], 
                   chem_data, 
                   raw_data,
                   nr=7,
                   nz=12, 
                   k_data=k_data,
                   chem_type=chem_type,
                   water_type=water_type,
                   optimize=False
                   )

print(column.k_data)

# column.model_uncertainty(capacity='None')
column.model_uncertainty()
## model_uncertainty now stores model results directly to column object, default is capacity=10, which runs +/-10% in capacity parameters

fix, ax = plt.subplots(1,1)

for i in range(len(column.data_df.columns)):
    sub_data = column.data_df[column.data_df.columns[i]]
    # print(sub_data)
    if 'influent' in column.data_df.columns[i]:
        ls=':'
        marker='x'
    else:
        ls='None'
        marker='.'
    ax.plot(sub_data.index,
             sub_data.values,
             ls=ls,
             marker=marker)

ax.plot(column.results.index,
         column.results.values,
         ls='-',
         marker='None',
         color='black')

# print(column.results)
# print(column.uncertainty_results)

ax.fill_between(column.uncertainty_results.index,
                column.uncertainty_results['lower'].values.astype('float64').T[0],
                column.uncertainty_results['upper'].values.astype('float64').T[0],
                color='grey',
                alpha=0.5)

plt.ylabel('Concentration ($\mu$g/L)')
plt.xlabel('Time (days)')

### NOTE: direct use of '.values' only works for this example because there is only a single compound. Otherwise, compound must be specified individually.


# ALTERNATE OPTIMIZER
Depending on selections this can take 1+ hours per compound !!!!

Instead of:

column.xn_range = np.linspace(.3,.75,19) # increments by 0.025

column.run_all_smart(pm=30, des_xn=0.01)
            
            checks +/- 30% of estimated K (pm=30), 
            and scans 1/n space in 0.01 step sizes (des_xn=0.01)

in previous snippet.

Use:

column.xn_range = np.linspace(0.3, 0.75, 46) 

            46 = interval 0.01 (19 is 0.025, for 0.3-0.75)
            
column.test_range = np.linspace(1, 5, 41)

            41 gives you 0.1 increment on K_multiplier
            K would not be expected to be lower than 1, but upper 
            bound of 3 to 4 may be sufficient to capture any real 
            values
    
Then include:

column.run_all()
        
        or

column.run_all(optimize='brute') #run only this line or the line above, not both

best fits are again stored back to 'column.k_data'

    
run_all() by default uses a multi-staged brute force approach. This subdivides the space into coarse grids, and performs a series of refining steps. 

'optimizer=brute' searches every combination of xn_range and test_range, while the default 'staged' behavior attempts to quickly locate the minimum by taking a coarser search while seeking the minimum. 
    
including 'optimize=brute' could be used after run_all_smart() or the staged approach to ensure that the K & 1/n values around a minimum were thoroughly searched.
    
## Performance: 
                brute (1+ hours per compound)
                staged (~5-10+ minutes per compound, depending on complexity)
                using run_all_smart (~2-15 minutes per compound)
  


# Final Notes: 
If you got this far, you may be interested in using this for 
larger datasets. In the above examples, each of the examples is set up
for looped execution. If you provide datasets with more than one compound
in the data sheet (and associated properties) then this will loop over each
compound individually.
